# Color Matrices from datasheet values for the CMV12000

In this notebook we are going to try to calculate the color matrecies for the dng development of the CMOSIS cmv12000 from the spectral response found in the datasheet.
For this we have extracted the data from the plots on page 19 of the [datasheet](data_raw/CMV12000_DS000603_5-00.pdf) as csv into the [all.csv](data/all.csv) file using a mixture of inkscape,
[this plugin](https://github.com/camrbuss/nodes_to_csv) and then rescaling the data with a python script to match the real units.

In [12]:
from email.base64mime import header_length
import numpy as np
import colour
x, *channels = np.transpose(np.genfromtxt("data/all.csv", delimiter=",", skip_header=1))
red, green_red, green_blue, blue, *_ = [colour.SpectralDistribution(channel, x).align(colour.SpectralShape(380, 730, 10)) for channel in channels]
print(f"loaded spectral data from {red.wavelengths[0]}nm to {red.wavelengths[-1]}nm")

loaded spectral data from 380.0nm to 730.0nm


We simulate two photographs of a colorchecker under two different iluminants A and D65. Then we generate a color matrix from the simulated exposures. The selection of the colorchecker might not be optimal but it has the advantage, that the experiment can easily be replicated in real life to check the plausibility of the results. In the future we might want to use different color targets that focus more on actual natural things like skin tones and foliage. For that we would need a dataset with spectral responeses of that though.

In [ ]:
def print_rust_matrix(matrix):
    for row in matrix:
        print(" ".join(f"({f:d}, 10000), " for f in row))

In [29]:
color_checker = colour.SDS_COLOURCHECKERS['BabelColor Average']

for iluminant in ['A', 'D65']:
    xyz_data = []
    raw_data = []
    for target in color_checker.data.keys():
        target_spectrum =  color_checker[target] * colour.SDS_ILLUMINANTS[iluminant].align(colour.SpectralShape(380, 730, 10))
        xyz_data.append(colour.colorimetry.sd_to_XYZ(target_spectrum))
        raw_data.append([np.sum((target_spectrum * channel).values) for channel in [red, green_red, blue]])
    
    raw_data_normalized = np.array(raw_data) / np.max(raw_data) * 10_000  # this scaling connstant is there to get a matrix in a reasonable range

    print(f"{iluminant} raw -> XYZ:")
    print(colour.matrix_colour_correction(raw_data_normalized, xyz_data))
    print(f"{iluminant} XYZ -> raw:")
    print(colour.matrix_colour_correction(xyz_data, raw_data_normalized))
    print()

A raw -> XYZ:
[[ 0.07445245  2.27973128 -1.38478893]
 [-0.45117767  3.06046338 -1.29881984]
 [-0.23271979 -0.42739597  3.09627007]]
A XYZ -> raw:
[[ 2.56510234 -1.7853895   0.31831867]
 [ 0.46871988  0.03087444  0.19762582]
 [ 0.25148638 -0.12230888  0.37033067]]

D65 raw -> XYZ:
[[ 0.2255407   0.65571105 -0.04778702]
 [-0.11276934  1.25917394 -0.29162897]
 [-0.07630574 -0.34850371  1.7361268 ]]
D65 XYZ -> raw:
[[ 3.21233103 -1.64519507 -0.21142885]
 [ 0.29065543  0.70213706  0.11869433]
 [ 0.1779698   0.08911394  0.59002806]]

